***Go Mind***

In [1]:
!pip install gymnasium torch tqdm matplotlib
!pip install pygame


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import matplotlib.pyplot as plt
from torch.distributions import Categorical
from tqdm import tqdm

In [ ]:
import pygame
import sys

BOARD_SIZE = 19
CELL_SIZE = 30
PADDING = 40
STONE_RADIUS = 12

EMPTY = 0
BLACK = 1
WHITE = 2

IS_DRAWN = True
HAS_PLAYER = True

if IS_DRAWN:
    pygame.init()
    screen = pygame.display.set_mode(
        (BOARD_SIZE * CELL_SIZE + PADDING * 2,
        BOARD_SIZE * CELL_SIZE + PADDING * 2)
    )
    pygame.display.set_caption("Go 19x19 (Interactive)")
    font = pygame.font.SysFont(None, 28)


def neighbors(x, y):
    for nx, ny in [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        if 0 <= nx < BOARD_SIZE and 0 <= ny < BOARD_SIZE:
            yield nx, ny


def get_group(board, x, y):
    """Возвращает группу камней и их свободы"""
    color = board[y][x]
    visited = set()
    group = set()
    liberties = set()

    stack = [(x, y)]
    while stack:
        cx, cy = stack.pop()
        if (cx, cy) in visited:
            continue
        visited.add((cx, cy))
        group.add((cx, cy))

        for nx, ny in neighbors(cx, cy):
            if board[ny][nx] == EMPTY:
                liberties.add((nx, ny))
            elif board[ny][nx] == color:
                stack.append((nx, ny))

    return group, liberties


def remove_dead_stones(board, color):
    """Удаляет группы указанного цвета без свобод"""
    removed = False
    to_remove = []

    for y in range(BOARD_SIZE):
        for x in range(BOARD_SIZE):
            if board[y][x] == color:
                group, libs = get_group(board, x, y)
                if len(libs) == 0:
                    to_remove.extend(group)
                    removed = True

    for x, y in to_remove:
        board[y][x] = EMPTY

    return removed


def is_suicide(board, x, y, color):
    """Проверка, что ход не самоубийственный"""
    temp_board = [row[:] for row in board]
    temp_board[y][x] = color

    # Сначала проверяем, у группы есть свободы
    _, libs = get_group(temp_board, x, y)
    if len(libs) > 0:
        return False

    # Если нет свобод — возможно, мы убиваем соперника → ход разрешён
    enemy = WHITE if color == BLACK else BLACK
    removed = False

    for nx, ny in neighbors(x, y):
        if temp_board[ny][nx] == enemy:
            g, libs = get_group(temp_board, nx, ny)
            if len(libs) == 0:
                removed = True

    return not removed


def try_place_stone(board, x, y, color):
    if board[y][x] != EMPTY:
        return False

    if is_suicide(board, x, y, color):
        return False

    # Ставим камень
    board[y][x] = color

    # Удаляем убитые группы противника
    enemy = WHITE if color == BLACK else BLACK
    remove_dead_stones(board, enemy)

    return True


def draw_board(board, turn):
    screen.fill((210, 170, 110))

    # Линии
    for i in range(BOARD_SIZE):
        pygame.draw.line(
            screen,
            (0, 0, 0),
            (PADDING, PADDING + i * CELL_SIZE),
            (PADDING + (BOARD_SIZE - 1) * CELL_SIZE, PADDING + i * CELL_SIZE),
            1
        )
        pygame.draw.line(
            screen,
            (0, 0, 0),
            (PADDING + i * CELL_SIZE, PADDING),
            (PADDING + i * CELL_SIZE, PADDING + (BOARD_SIZE - 1) * CELL_SIZE),
            1
        )

    # Хоcи (точки)
    hoshi = [3, 9, 15]
    for hx in hoshi:
        for hy in hoshi:
            pygame.draw.circle(
                screen, (0, 0, 0),
                (PADDING + hx * CELL_SIZE, PADDING + hy * CELL_SIZE),
                4
            )

    # Камни
    for y in range(BOARD_SIZE):
        for x in range(BOARD_SIZE):
            if board[y][x] == BLACK:
                pygame.draw.circle(
                    screen,
                    (0, 0, 0),
                    (PADDING + x * CELL_SIZE, PADDING + y * CELL_SIZE),
                    STONE_RADIUS
                )
            elif board[y][x] == WHITE:
                pygame.draw.circle(
                    screen,
                    (255, 255, 255),
                    (PADDING + x * CELL_SIZE, PADDING + y * CELL_SIZE),
                    STONE_RADIUS
                )

    turn_text = "Ход: Черные" if turn == BLACK else "Ход: Белые"
    t = font.render(turn_text, True, (0, 0, 0))
    screen.blit(t, (10, 10))

    pygame.display.flip()

def player_step(board, event):
    if event.type ==  event.type == pygame.MOUSEBUTTONDOWN:
        mx, my = event.pos

        # Находим ближайшую точку пересечения
        x = round((mx - PADDING) / CELL_SIZE)
        y = round((my - PADDING) / CELL_SIZE)

        if 0 <= x < BOARD_SIZE and 0 <= y < BOARD_SIZE:
            return x, y
    return -1, -1

def ai_step():


def main():
    board = [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    turn = BLACK

    clock = pygame.time.Clock()

    while True:
        if HAS_PLAYER:
            draw_board(board, turn)

            x, y = None, None
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if turn == BLACK and HAS_PLAYER:
                    x, y = player_step(board, event)
                    if x == y == -1:
                        continue
            if x == y == None:
                x, y = ai_step(board, turn)
            if try_place_stone(board, x, y, turn):
                turn = WHITE if turn == BLACK else BLACK
        else:


            

        clock.tick(30)


main()


SystemExit: 

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
